In [8]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [3]:
!wget -O BCCD_Dataset.zip https://github.com/Shenggan/BCCD_Dataset/archive/refs/heads/master.zip
!unzip BCCD_Dataset.zip -d BCCD_Dataset

--2025-03-17 09:56:04--  https://github.com/Shenggan/BCCD_Dataset/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Shenggan/BCCD_Dataset/zip/refs/heads/master [following]
--2025-03-17 09:56:05--  https://codeload.github.com/Shenggan/BCCD_Dataset/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘BCCD_Dataset.zip’

BCCD_Dataset.zip        [  <=>               ]   7.56M  20.6MB/s    in 0.4s    

2025-03-17 09:56:05 (20.6 MB/s) - ‘BCCD_Dataset.zip’ saved [7927884]

Archive:  BCCD_Dataset.zip
d272fb14cdff6e473fafeeeba32aba5f560e9e43
   creating: BCCD_Dataset/BCCD_Dataset-master/
 ex

In [9]:
import os

# Create necessary folders for YOLOv10 dataset format
folders = [
    'dataset/images/train', 'dataset/images/val', 'dataset/images/test',
    'dataset/labels/train', 'dataset/labels/val', 'dataset/labels/test'
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

In [10]:
import xmltodict
import shutil
from tqdm import tqdm

# Function to convert VOC XML to YOLO format
def convert_voc_to_yolo(xml_file, img_width, img_height):
    with open(xml_file) as f:
        data = xmltodict.parse(f.read())

    objects = data['annotation'].get('object', [])
    if not isinstance(objects, list):
        objects = [objects]

    yolo_data = []
    for obj in objects:
        class_id = {"WBC": 0, "RBC": 1, "Platelets": 2}.get(obj['name'], -1)
        if class_id == -1:
            continue

        bbox = obj['bndbox']
        x_center = (float(bbox['xmin']) + float(bbox['xmax'])) / 2 / img_width
        y_center = (float(bbox['ymin']) + float(bbox['ymax'])) / 2 / img_height
        width = (float(bbox['xmax']) - float(bbox['xmin'])) / img_width
        height = (float(bbox['ymax']) - float(bbox['ymin'])) / img_height

        yolo_data.append(f"{class_id} {x_center} {y_center} {width} {height}")
    return yolo_data

# Function to prepare dataset
def prepare_dataset(file_list, img_split):
    for img_name in tqdm(file_list):
        # Copy images
        shutil.copy(f'/content/drive/MyDrive/BCCD_Dataset/BCCD_Dataset-master/BCCD/JPEGImages/{img_name}.jpg',
                    f'/content/drive/MyDrive/dataset/images/{img_split}/{img_name}.jpg')

        # Convert and copy labels
        xml_file = f'/content/drive/MyDrive/BCCD_Dataset/BCCD_Dataset-master/BCCD/Annotations/{img_name}.xml'
        if os.path.exists(xml_file):
            with open(f'/content/drive/MyDrive/dataset/labels/{img_split}/{img_name}.txt', 'w') as label_file:
                img_width, img_height = 640, 640
                for line in convert_voc_to_yolo(xml_file, img_width, img_height):
                    label_file.write(line + '\n')

# Load dataset split
with open('/content/drive/MyDrive/BCCD_Dataset/BCCD_Dataset-master/BCCD/ImageSets/Main/train.txt') as f:
    train_images = f.read().strip().split()

with open('/content/drive/MyDrive/BCCD_Dataset/BCCD_Dataset-master/BCCD/ImageSets/Main/val.txt') as f:
    val_images = f.read().strip().split()

with open('/content/drive/MyDrive/BCCD_Dataset/BCCD_Dataset-master/BCCD/ImageSets/Main/test.txt') as f:
    test_images = f.read().strip().split()

# Organize data into train/val/test folders
prepare_dataset(train_images, 'train')
prepare_dataset(val_images, 'val')
prepare_dataset(test_images, 'test')


100%|██████████| 72/72 [00:29<00:00,  2.44it/s]


In [11]:
with open('dataset/data.yaml', 'w') as f:
    f.write("""\
path: ../dataset
train: images/train
val: images/val
test: images/test

names:
  0: WBC
  1: RBC
  2: Platelets
""")


In [13]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [14]:
from ultralytics import YOLO

# Load the YOLOv10 model
model = YOLO('yolov10n.pt')

# Train the model
model.train(data='dataset/data.yaml', epochs=50, imgsz=640, batch=16)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.59M/5.59M [00:00<00:00, 69.2MB/s]


Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov10n.pt, data=dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, l

100%|██████████| 755k/755k [00:00<00:00, 14.9MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1      9856  ultralytics.nn.modules.block.SCDown          [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1     36096  ultralytics

...

                   all         87       1138      0.711      0.741      0.778      0.527



50 epochs completed in 0.098 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.7MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.7MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10n summary (fused): 125 layers, 2,695,586 parameters, 0 gradients, 8.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.40it/s]


                   all         87       1138      0.711      0.739      0.778      0.527
                   WBC         86         87      0.836      0.999      0.974      0.766
                   RBC         83        968       0.67      0.755      0.782      0.544
             Platelets         45         83      0.626      0.463      0.577      0.271
Speed: 0.3ms preprocess, 4.0ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fbadc2dff90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [15]:
results = model.predict(source='dataset/images/test/', save=True)


image 1/72 /content/drive/MyDrive/dataset/images/test/BloodImage_00007.jpg: 480x640 1 WBC, 8 RBCs, 44.3ms
image 2/72 /content/drive/MyDrive/dataset/images/test/BloodImage_00011.jpg: 480x640 1 WBC, 11 RBCs, 10.2ms
image 3/72 /content/drive/MyDrive/dataset/images/test/BloodImage_00015.jpg: 480x640 2 WBCs, 12 RBCs, 10.0ms
image 4/72 /content/drive/MyDrive/dataset/images/test/BloodImage_00016.jpg: 480x640 1 WBC, 9 RBCs, 17.8ms
image 5/72 /content/drive/MyDrive/dataset/images/test/BloodImage_00018.jpg: 480x640 1 WBC, 17 RBCs, 10.0ms
image 6/72 /content/drive/MyDrive/dataset/images/test/BloodImage_00019.jpg: 480x640 1 WBC, 9 RBCs, 9.9ms
image 7/72 /content/drive/MyDrive/dataset/images/test/BloodImage_00021.jpg: 480x640 1 WBC, 14 RBCs, 2 Plateletss, 9.9ms
image 8/72 /content/drive/MyDrive/dataset/images/test/BloodImage_00031.jpg: 480x640 3 WBCs, 11 RBCs, 3 Plateletss, 10.1ms
image 9/72 /content/drive/MyDrive/dataset/images/test/BloodImage_00033.jpg: 480x640 1 WBC, 10 RBCs, 10.6ms
image 10/72

In [16]:
metrics = model.val(data='dataset/data.yaml')
print(metrics)

Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


val: Scanning /content/drive/MyDrive/dataset/labels/val.cache... 87 images, 0 backgrounds, 0 corrupt: 100%|██████████| 87/87 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.73it/s]


                   all         87       1138       0.71      0.741      0.778      0.528
                   WBC         86         87      0.836      0.995      0.974      0.768
                   RBC         83        968      0.668      0.757      0.782      0.544
             Platelets         45         83      0.627       0.47      0.578      0.271
Speed: 5.1ms preprocess, 7.0ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/train3
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fbbdae1ad90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,   